This notebook will setup the trajectories used in benchmarking. It also demonstrates how to upload a `.zarrtraj` trajectory
to an S3 bucket directly using `Zarr` and `s3fs` without use of the ZarrTrajWriter

1. Download the 90ns YiiP trajectory to the local filesystem
2. Write it into the zarrtraj format on disk
3. Upload the zarrtraj-formatted trajectory to an accessible AWS S3 bucket

In [ ]:
import zarrtraj
import MDAnalysis as mda
import MDAnalysisData
import zarr

# 1
yiip = MDAnalysisData.yiip_equilibrium.fetch_yiip_equilibrium_long(data_home='notebook_data_tmp')
# 2
u = mda.Universe(yiip.topology, yiip.trajectory)
z = zarr.open_group("notebook_data_tmp/yiip.zarrtraj", mode = 'w')
with mda.Writer(z, u.atoms.n_atoms,
                    # Select _ frames 
                    chunks=(10, u.trajectory.n_atoms, 3),
                    format='ZARRTRAJ') as W:
    for ts in u.trajectory:
        W.write(u.atoms)

In [ ]:
import os 
import s3fs
import zarr

# 3
# Must set the environmental variables using Jupyter's directive:
# "%env KEY=VALUE"
# before running

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("BUCKET_NAME")

source = zarr.open_group("notebook_data_tmp/yiip.zarrtraj", mode='r')

s3_fs = s3fs.S3FileSystem(
    # anon must be false to allow authentication
    anon=False,
    client_kwargs=dict(
        region_name='us-east-1',
    )
)

cloud_store = s3fs.S3Map(
    root=f'{BUCKET_NAME}/yiip.zarrtraj',
    s3=s3_fs,
    check=False
)

zarr.convenience.copy_store(source.store, cloud_store,
                            if_exists='replace')